In [1]:
import tensorflow as tf
import numpy as np
from bregman.suite import *

In [2]:
k = 2
segment_size = 50
max_iterations = 100

In [3]:
chroma = tf.placeholder(tf.float32)
max_freqs = tf.argmax(chroma, 0)

In [4]:
def get_chromagram(audio_file):
    F = Chromagram(audio_file, nfft=16384, wfft=8192, nhop=2205)
    return F.X

In [5]:
def get_dataset(sess, audio_file):
    chroma_data = get_chromagram(audio_file)
    print('chroma_data', np.shape(chroma_data))
    chroma_length = np.shape(chroma_data)[1]
    print('chroma_length', chroma_length)
    xs = []
    for i in range(chroma_length / segment_size):
        chroma_segment = chroma_data[:, i*segment_size:(i+1)*segment_size]
        x = extract_feature_vector(sess, chroma_segment)
        if len(xs) == 0:
            xs = x
        else:
            xs = np.vstack((xs, x))
    return xs

In [6]:
def extract_feature_vector(sess, chroma_data):
    num_features, num_samples = np.shape(chroma_data)
    freq_vals = sess.run(max_freqs, feed_dict={chroma: chroma_data})
    hist, bins = np.histogram(freq_vals, bins=range(num_features + 1))
    return hist.astype(float) / num_samples

In [7]:
def initial_cluster_centroids(X, k):
    return X[0:k, :]

In [8]:
def recompute_centroids(X, Y):
    sums = tf.unsorted_segment_sum(X, Y, k)
    counts = tf.unsorted_segment_sum(tf.ones_like(X), Y, k)
    return sums / counts

In [9]:
def assign_cluster(X, centroids):
    expanded_vectors = tf.expand_dims(X, 0)
    expanded_centroids = tf.expand_dims(centroids, 1)
    distances = tf.reduce_sum(tf.square(tf.subtract(expanded_vectors, expanded_centroids)), 2)
    mins = tf.argmin(distances, 0)
    return mins

In [10]:
with tf.Session() as sess:
    X = get_dataset(sess, 'TalkingMachinesPodcast.wav')
    print(np.shape(X))
    centroids = initial_cluster_centroids(X, k)
    i, converged = 0, False
    while not converged and i < max_iterations:
        i += 1
        Y = assign_cluster(X, centroids)
        centroids = sess.run(recompute_centroids(X, Y))
        if i % 50 == 0:
            print('iteration', i)
            
    segments = sess.run(Y)
    for i in range(len(segments)):
        seconds = (i * segment_size) / float(10)
        min, sec = divmod(seconds, 60)
        time_str = '{}m {}s'.format(min, sec)
        print(time_str, segments[i])

('chroma_data', (12, 633))
('chroma_length', 633)
(12, 12)
('iteration', 50)
('iteration', 100)
('0.0m 0.0s', 0)
('0.0m 5.0s', 1)
('0.0m 10.0s', 0)
('0.0m 15.0s', 1)
('0.0m 20.0s', 1)
('0.0m 25.0s', 1)
('0.0m 30.0s', 1)
('0.0m 35.0s', 0)
('0.0m 40.0s', 1)
('0.0m 45.0s', 1)
('0.0m 50.0s', 0)
('0.0m 55.0s', 0)
